In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

In [11]:
#importamos la tabla de join 
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')
display(join.columns)
#holi

Index(['RowID', 'Fecha', 'IDItem', 'NumeroUbicacion', 'Dewey', 'Ubicacion',
       'Llave', 'Programa', 'Facultad', 'IDUsuario', 'Year', 'Signatura',
       'TipoItem', 'Autor', 'Titulo', 'AnioPublicacion', 'DeweyEspecifico',
       'TemaDewey', 'DeweyUnidad', 'DeweyDecena', 'DeweyCentena', 'Temas',
       'Union'],
      dtype='object')

In [12]:
#Ejemplos y columnas del dataframe
display(join.head(3))
display(join.columns)

,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,Facultad,IDUsuario,...,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena,Temas,Union
0,Row0,1547050497000,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,...,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,720,720,700,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,1547050500000,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,...,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,720,720,700,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,1547138763000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,...,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,540,540,500,[quimic],quimic


Index(['RowID', 'Fecha', 'IDItem', 'NumeroUbicacion', 'Dewey', 'Ubicacion',
       'Llave', 'Programa', 'Facultad', 'IDUsuario', 'Year', 'Signatura',
       'TipoItem', 'Autor', 'Titulo', 'AnioPublicacion', 'DeweyEspecifico',
       'TemaDewey', 'DeweyUnidad', 'DeweyDecena', 'DeweyCentena', 'Temas',
       'Union'],
      dtype='object')

In [11]:
#Creamos la columna pesos para el dataframe a partir del año en que se realizó el prestamo
join["Peso"] = join.apply(lambda row: 1/2**(2021-row.Year), axis=1 )

In [26]:
#Esta funcion crea la tabla de pesos
#parámetros
    #columna = {deweyUnidad|deweyDecena|deweyCentena}
    #nombreArchivo = nombre con el que se va a exportar el archivo
def crearTablaPesos(columna, nombreArchivo):
    #Tabla = pesos_usuarios
        #columnas = deweys
        #filas = usuarios
        #dato = peso que tiene el usuario en dicho dewey
    pesos_usuarios = pd.DataFrame(columns= join[columna].unique(),dtype=float)
    tam = len(join[columna].unique())
    pesos_usuarios.columns = pesos_usuarios.columns.astype(str)
    display(pesos_usuarios)

    ids = join["IDUsuario"].unique()
    i = 0
    for usuario in ids:
      pesos_usuarios.loc[i] = np.zeros((tam,), dtype=int)
      #Para cada usuario traer sus Deweys y sus pesas asociados
      prestamos = join.loc[join["IDUsuario"]==usuario]
      deweys = prestamos[[columna,"Peso"]]
      result_user = deweys.groupby(columna)["Peso"].sum().reset_index(name="Peso")
      # Recorrer los Deweys y Pesos para plasmarlos en una matriz
      #display(result_user)
      for index, row in result_user.iterrows():
        d = row[columna]
        p=row['Peso']
        pesos_usuarios[str(int(d))][i] = p
      i = i+1
    ruta = r'C:/Users/johan/Desktop/GitKraken/DataOutOfGit/Limpieza/'
    ruta = ruta + nombreArchivo
    pesos_usuarios.to_json(ruta)

In [25]:
crearTablaPesos("DeweyUnidad", "pesos_usuario_x_dewey_unidad.json")

,720,540,512,712,741,814,823,330,863,696,...,267,114,34,673,447,600,653,263,91,586


,720,540,512,712,741,814,823,330,863,696,...,267,114,34,673,447,600,653,263,91,586
0,2.00,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125
1,0.00,1.0,1.25,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
2,0.00,0.0,0.00,3.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
3,0.25,0.0,0.00,0.000,0.25,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
4,5.00,0.0,0.00,0.875,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24402,0.00,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
24403,0.00,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
24404,0.00,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
24405,0.00,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000


In [27]:
crearTablaPesos("DeweyDecena", "pesos_usuario_x_dewey_decena.json")

,720,540,510,710,740,810,820,330,860,690,...,60,200,480,960,310,280,50,90,990,30


In [28]:
crearTablaPesos("DeweyCentena", "pesos_usuario_x_dewey_centena.json")

,700,500,800,300,600,100,900,-900,400,0,200


In [3]:
pesos_usuarios_unidad = pd.read_json('https://www.dropbox.com/s/oliptl0pv39bkjm/pesos_usuario_x_dewey_unidad.json?dl=1')

In [4]:
pesos_usuarios_decena = pd.read_json('https://www.dropbox.com/s/33xvh0xderxu1ss/pesos_usuario_x_dewey_decena.json?dl=1')

In [5]:
pesos_usuarios_centena = pd.read_json('https://www.dropbox.com/s/1gj7u98rozx6i72/pesos_usuario_x_dewey_centena.json?dl=1')

In [9]:
pesos_usuarios_unidad['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_unidad.head(3)

,720,540,512,712,741,814,823,330,863,696,...,114,34,673,447,600,653,263,91,586,IDUsuario
0,2.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,c361c772cd0220bb16dcfb2ad803e4903334ab94
1,0.0,1.0,1.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,87b0e5a61ed712ddfaf5d478ad68c87c825997e9
2,0.0,0.0,0.00,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,afe8c17ba351d274fc5671978918f0c93032d88f


In [62]:
def normalizar_pesos(pesos_usuario):
    
    if len(pesos_usuario) == len(join.IDUsuario.unique()):
        pesos_usuario['IDUsuario'] = join.IDUsuario.unique()
                                 
    copia = pesos_usuario.copy()
    copia = copia.drop(labels = ["IDUsuario"], axis = 1)
    sumatoria = copia.sum(axis=1)
    pesos_norm = copia.div(sumatoria, axis=0).fillna(0)
    pesos_norm_id = pesos_norm.copy()
    
    if len(pesos_norm_id) == len(join.IDUsuario.unique()):
        pesos_norm_id['IDUsuario'] = join.IDUsuario.unique()
    
    return pesos_norm_id

In [11]:
def eliminar_cero(id_usuario, df_pesos):
    m1 = (df_pesos['IDUsuario'] == id_usuario)
    m2 = (df_pesos[m1] != 0).all()
    return df_pesos.loc[m1,m2]

In [12]:
pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios_unidad)
display(pesos_norm_id_unidad.shape[0])

24407

In [13]:
pesos_norm_id_decena = normalizar_pesos(pesos_usuarios_decena)
display(pesos_norm_id_decena.shape[0])

24407

In [14]:
pesos_norm_id_centena = normalizar_pesos(pesos_usuarios_centena)
display(pesos_norm_id_centena.shape[0])

24407

In [16]:
display(eliminar_cero('c361c772cd0220bb16dcfb2ad803e4903334ab94',pesos_usuarios_unidad))

,720,711,725,726,428,746,155,770,378,589,771,584,582,586,IDUsuario
0,2.0,0.25,0.125,0.5,0.5,0.25,0.25,0.25,0.5,0.125,0.125,0.125,0.125,0.125,c361c772cd0220bb16dcfb2ad803e4903334ab94


In [17]:
def clustering(df_pesos, treshold, nombre_archivo):
    i = 0
    resultados = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia','Peso'])
    for usuario in df_pesos.IDUsuario.unique():
        usuario_limpio = eliminar_cero(usuario, df_pesos)
        usuario_limpio = usuario_limpio.drop(labels = 'IDUsuario', axis = 1)
        for column in usuario_limpio:
            if usuario_limpio.iloc[0][column] >= treshold:
                peso = df_pesos.loc[df_pesos.IDUsuario == usuario][column]
                #print(peso.values[0])
                resultados.loc[i] = np.array([usuario, column, usuario_limpio.iloc[0][column],peso.values[0]])
                i = i + 1
    '''
    ruta = r'C:/Users/johan/Desktop/GitKraken/DataOutOfGit/Limpieza/'
    ruta = ruta + nombreArchivo
    resultados.to_json(ruta)
    '''
    return resultados

In [ ]:
pesos_clustering = clustering(pesos_norm_id_unidad, 0, 'pesos_clustering_unidad.json')
display(pesos_clustering)

In [59]:
pesos_clustering.to_json(r'C:\Users\user\Downloads\pesos_clustering.json')

In [18]:
pesos_clustering = pd.read_json('https://www.dropbox.com/s/3clldbh4bv3v4m8/pesos_clustering.json?dl=1')

In [22]:
display(pesos_clustering.head(3))
print(pesos_clustering.shape)

,IDUsuario,Cluster,Pertenencia,Peso
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,720,0.380952,2.000
1,c361c772cd0220bb16dcfb2ad803e4903334ab94,711,0.047619,0.250
2,c361c772cd0220bb16dcfb2ad803e4903334ab94,725,0.023810,0.125


(125653, 4)


In [23]:
from sklearn import preprocessing
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')
display(join.head(3))

,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,Facultad,IDUsuario,...,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena,Temas,Union
0,Row0,1547050497000,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,...,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,720,720,700,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,1547050500000,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,...,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,720,720,700,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,1547138763000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,...,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,540,540,500,[quimic],quimic


In [24]:
def rank_material(pesos_clustering, cluster, Dewey_XD):
    #Buscamos únicamente a los usuarios que tienen un grado de pertenencia al cluster.
    usuarios = pesos_clustering.loc[pesos_clustering.Cluster == str(cluster)]
    usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
    usuarios.loc[usuarios.Peso > 10, "Peso"] = 10
    #normalizar columna de peso
    max_value = usuarios["Peso"].max()
    min_value = usuarios["Peso"].min()
    usuarios["Peso"] = (usuarios["Peso"]) / (max_value)
   #filtramos de la tabla de join los rows con el dewey
    #prestamos = pesos_clustering.loc[join.Signatura == cluster]
    prestamos = join.loc[join[Dewey_XD] == cluster]
    #join
    prestamos_data = pd.DataFrame(data=prestamos)
    usuario_data = pd.DataFrame(data=usuarios)
    join_tablas = pd.merge(prestamos_data, usuario_data, left_on='IDUsuario', right_on='IDUsuario', how='inner')
    materialRank = join_tablas.groupby(["Llave"])["Peso"].sum().reset_index(name="Peso")
    materialRank = materialRank.sort_values(by=["Peso"],ascending=False)
    return materialRank


In [179]:
lista_recomendaciones = rank_material("720")

TypeError: rank_material() missing 2 required positional arguments: 'cluster' and 'Dewey_XD'

In [ ]:
pesos_norm_id = normalizar_pesos(pesos_usuarios_unidad)
pesos_norm_id_cluster = clustering(pesos_norm_id, 0, "")

In [282]:
def generar_recomendacion(pesos_clustering, cluster, peso_x_nivel, total_recomendaciones, material_rankeado):
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
    
    prestamos_cluster = pesos_clustering.loc[pesos_clustering.Cluster == str(cluster)]
    prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float) 
    for index, row in prestamos_cluster.iterrows():
        num_prestamos = round(row["Pertenencia"] * peso_x_nivel * total_recomendaciones)
        #print(num_prestamos)
        #print("Recomendaciones para el usuario:", row["IDUsuario"])
        aux = material_rankeado.head(num_prestamos)
        #print(aux.Llave)
        for llave in aux.Llave.iteritems():
            aux_df = pd.DataFrame({'IDUsuario': row.IDUsuario, 'Llave': llave[1], 'Nivel': peso_x_nivel}, index = ["Index 1"])
            #print("Aux: ", aux_df)
            recomendaciones = pd.concat([recomendaciones, aux_df], ignore_index = True)
            #print("Recomendaciones del cluster: ", cluster, recomendaciones.shape[0])
    return recomendaciones

In [288]:
def recomendaciones_nivel(df_pesos, peso_x_nivel, total_recomendaciones):
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
    pesos_norm_id = normalizar_pesos(df_pesos)
    pesos_norm_id_cluster = clustering(pesos_norm_id, 0, "")
    for cluster in df_pesos:
        material_rankeado = rank_material(pesos_norm_id_cluster, cluster, "DeweyUnidad")
        #print(material_rankeado)
        lista_recomendaciones = generar_recomendacion(pesos_norm_id_cluster, cluster, peso_x_nivel, total_recomendaciones, material_rankeado)
        #print(lista_recomendaciones.shape[0])
        recomendaciones = pd.concat([recomendaciones, lista_recomendaciones], ignore_index = True)
        print("Recomendaciones para el grupo: ", cluster, lista_recomendaciones.shape[0])
    return recomendaciones

In [270]:
recomendaciones_final = recomendaciones_nivel(pesos_usuarios_unidad, 0.7, 10)

<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  720 1906


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  540 314


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

...

<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  114 3
Recomendaciones para el grupo:  34 1
Recomendaciones para el grupo:  673 0
Recomendaciones para el grupo:  447 2
Recomendaciones para el grupo:  600 0


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  653 0
Recomendaciones para el grupo:  263 0
Recomendaciones para el grupo:  91 0
Recomendaciones para el grupo:  586 0
Recomendaciones para el grupo:  IDUsuario 0


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [271]:
display(recomendaciones_final)

,IDUsuario,Llave,Nivel
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,312423,0.7
1,c361c772cd0220bb16dcfb2ad803e4903334ab94,905978,0.7
2,c361c772cd0220bb16dcfb2ad803e4903334ab94,312423,0.7
3,6b39c8ff51a36458e3f68233b82c1ce8bd8cc3d3,312423,0.7
4,6b39c8ff51a36458e3f68233b82c1ce8bd8cc3d3,905978,0.7
...,...,...,...
164585,3877f9e2db2bdc79ab581453f015ac952ef57bfa,795219,0.7
164586,b6e20ea077a61a92ccf28cdc8c815c66e8886b0f,263299,0.7
164587,bebf9632468bd8962692cb850fc461ba7260f026,745446,0.7
164588,ea158017be832823712035897de2f610c5f9ed2b,759132,0.7


In [289]:
recomendaciones_final_decena = recomendaciones_nivel(pesos_usuarios_decena, 0.2, 10)
display(recomendaciones_final_decena)

<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  720 898


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  540 194


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

...

Recomendaciones para el grupo:  480 3
Recomendaciones para el grupo:  960 18
Recomendaciones para el grupo:  310 10
Recomendaciones para el grupo:  280 11


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  50 0
Recomendaciones para el grupo:  90 0
Recomendaciones para el grupo:  990 0
Recomendaciones para el grupo:  30 0
Recomendaciones para el grupo:  IDUsuario 0


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,IDUsuario,Llave,Nivel
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,312423,0.2
1,6b39c8ff51a36458e3f68233b82c1ce8bd8cc3d3,312423,0.2
2,a8cd77c9b47b2723fbd8013ce467664f93ad73e6,312423,0.2
3,a8cd77c9b47b2723fbd8013ce467664f93ad73e6,312423,0.2
4,145f664af916e3bb70e66a166b3e13ce7b70b27b,312423,0.2
...,...,...,...
37936,8bb8779549518faaed830e3a97b8427d71ad8b17,273317,0.2
37937,8bb8779549518faaed830e3a97b8427d71ad8b17,274712,0.2
37938,7f383739a8fa2004c90196a376bd82fab0be196d,273317,0.2
37939,6ac71f94abab1d583c0f2c32631cf7093b1faee7,273317,0.2


In [290]:
recomendaciones_final_centena = recomendaciones_nivel(pesos_usuarios_centena, 0.1, 10)
display(recomendaciones_final_centena)

<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  700 2038


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  500 1536


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

...

Recomendaciones para el grupo:  0 0


<ipython-input-185-5639f87c2442>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-185-5639f87c2442>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recomendaciones para el grupo:  200 297
Recomendaciones para el grupo:  IDUsuario 0


C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,IDUsuario,Llave,Nivel
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,791686,0.1
1,afe8c17ba351d274fc5671978918f0c93032d88f,791686,0.1
2,6b39c8ff51a36458e3f68233b82c1ce8bd8cc3d3,791686,0.1
3,a8cd77c9b47b2723fbd8013ce467664f93ad73e6,791686,0.1
4,c4b92bc46c80a9e24b1cece78317ddb49310db2f,791686,0.1
...,...,...,...
18236,e6d0e0c13bab1a7fa8eab6f2ae9bcdf59128342b,346714,0.1
18237,90d586b80be17ec2179c5a752547d15da99ebb66,346714,0.1
18238,8e9db7f71e7c19c6f6abf15babc038ade2522424,346714,0.1
18239,243b1d77ee9fcf237c64bac9499fa40a5c74f372,346714,0.1


In [274]:
def recomendaciones_usuario(id_usuario, df_recomendaciones):
    recomendaciones_u = df_recomendaciones.loc[df_recomendaciones["IDUsuario"] == id_usuario]
    display(recomendaciones_u)

In [292]:
recomendaciones_tres_niveles = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
recomendaciones_tres_niveles = pd.concat([recomendaciones_tres_niveles, recomendaciones_final], ignore_index = True)
recomendaciones_tres_niveles = pd.concat([recomendaciones_tres_niveles, recomendaciones_final_decena], ignore_index = True)
recomendaciones_tres_niveles = pd.concat([recomendaciones_tres_niveles, recomendaciones_final_centena], ignore_index = True)

recomendaciones_usuario('c361c772cd0220bb16dcfb2ad803e4903334ab94', recomendaciones_tres_niveles)

,IDUsuario,Llave,Nivel
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,312423,0.7
1,c361c772cd0220bb16dcfb2ad803e4903334ab94,905978,0.7
2,c361c772cd0220bb16dcfb2ad803e4903334ab94,312423,0.7
47541,c361c772cd0220bb16dcfb2ad803e4903334ab94,259981,0.7
52129,c361c772cd0220bb16dcfb2ad803e4903334ab94,862239,0.7
134136,c361c772cd0220bb16dcfb2ad803e4903334ab94,289844,0.7
164590,c361c772cd0220bb16dcfb2ad803e4903334ab94,312423,0.2
202531,c361c772cd0220bb16dcfb2ad803e4903334ab94,791686,0.1


In [1]:
import os
os.system('jupyter nbconvert --to html ClusteringYRecomendacionesDifusas.ipynb')

0

In [13]:
pesos_usuarios_unidad['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_decena['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_centena['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_unidad.columns

Index([        720,         540,         512,         712,         741,
               814,         823,         330,         863,         696,
       ...
               114,          34,         673,         447,         600,
               653,         263,          91,         586, 'IDUsuario'],
      dtype='object', length=756)

In [71]:
def pesos_completos_usuario(id_usuario):
    pesos_unidad = pesos_usuarios_unidad.loc[pesos_usuarios_unidad["IDUsuario"] == id_usuario]
    pesos_decena = pesos_usuarios_decena.loc[pesos_usuarios_decena["IDUsuario"] == id_usuario]
    pesos_centena = pesos_usuarios_centena.loc[pesos_usuarios_centena["IDUsuario"] == id_usuario]
    
    pesos_unidad_copy = pesos_unidad.copy()
    pesos_decena_copy = pesos_decena.copy()
    pesos_centena_copy = pesos_centena.copy()
    
    pesos_unidad_copy = pesos_unidad_copy.drop(labels = ["IDUsuario"], axis = 1)
    pesos_decena_copy = pesos_decena_copy.drop(labels = ["IDUsuario"], axis = 1)
    pesos_centena_copy = pesos_centena_copy.drop(labels = ["IDUsuario"], axis = 1)
    
    #Top Deweys
    
    pesos_unidad_copy = pesos_unidad_copy.T
    pesos_decena_copy = pesos_decena_copy.T
    pesos_centena_copy = pesos_centena_copy.T
    
    pesos_unidad_copy = pesos_unidad_copy.sort_values(by=[0], ascending = False)
    pesos_decena_copy = pesos_decena_copy.sort_values(by=[0], ascending = False)
    pesos_centena_copy = pesos_centena_copy.sort_values(by=[0], ascending = False)
    
    pesos_unidad_copy = pesos_unidad_copy.T
    pesos_decena_copy = pesos_decena_copy.T
    pesos_centena_copy = pesos_centena_copy.T
    
    #Top Dewey normalizado
    
    pesos_unidad_norm = normalizar_pesos(pesos_unidad)
    pesos_decena_norm = normalizar_pesos(pesos_decena)
    pesos_centena_norm = normalizar_pesos(pesos_centena)
    
    pesos_unidad_norm_copy = pesos_unidad_norm.copy()
    pesos_decena_norm_copy = pesos_decena_norm.copy()
    pesos_centena_norm_copy = pesos_centena_norm.copy()
    
    pesos_unidad_norm_copy = pesos_unidad_norm_copy.T
    pesos_decena_norm_copy = pesos_decena_norm_copy.T
    pesos_centena_norm_copy = pesos_centena_norm_copy.T
    
    pesos_unidad_norm_copy = pesos_unidad_norm_copy.sort_values(by=[0], ascending = False)
    pesos_decena_norm_copy = pesos_decena_norm_copy.sort_values(by=[0], ascending = False)
    pesos_centena_norm_copy = pesos_centena_norm_copy.sort_values(by=[0], ascending = False)
    
    pesos_unidad_norm_copy = pesos_unidad_norm_copy.T
    pesos_decena_norm_copy = pesos_decena_norm_copy.T
    pesos_centena_norm_copy = pesos_centena_norm_copy.T
    '''
    pesos_unidad_copy['Total'] = pesos_unidad_copy.sum(axis = 1)
    pesos_decena_copy['Total'] = pesos_decena_copy.sum(axis = 1)
    pesos_centena_copy['Total'] = pesos_centena_copy.sum(axis = 1)
    '''
    
    return [pesos_unidad_copy, pesos_decena_copy, pesos_centena_copy, pesos_unidad_norm_copy, pesos_decena_norm_copy, pesos_centena_norm_copy]

In [72]:
pesos_usuario_test = pesos_completos_usuario('c361c772cd0220bb16dcfb2ad803e4903334ab94')
pd.set_option('max_columns',40)
display(pesos_usuario_test[3])
display(pesos_usuario_test[4])
display(pesos_usuario_test[5])

,720,378,428,726,711,770,155,746,582,584,771,589,725,586,728,458,637,544,535,587,...,362,814,677,537,717,757,599,871,348,950,266,636,593,830,695,590,159,438,341,324
0,0.380952,0.095238,0.095238,0.095238,0.047619,0.047619,0.047619,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,720,370,420,580,770,710,740,150,440,780,600,170,760,660,250,20,640,950,830,670,...,410,540,320,900,880,500,850,930,590,520,730,110,790,450,610,120,680,910,0,30
0,0.5,0.095238,0.095238,0.095238,0.071429,0.047619,0.047619,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,700,500,300,400,100,800,600,900,-900,0,200
0,0.666667,0.095238,0.095238,0.095238,0.047619,0.0,0.0,0.0,0.0,0.0,0.0
